# Case Study: Movie Data Analysis

This notebook uses a dataset from the MovieLens website.

* Data SourceL MovieLens web site (filename: ml-20m.zip)
* Location https://grouplens.org/datasets/movielens


First Let's explore the folder to see what files we have in the dataset

In [1]:
!ls -la ./movielens

total 855092
drwxr-xr-x 1 aydin 197609         0 Mar  5 23:15 .
drwxr-xr-x 1 aydin 197609         0 Mar 10 17:41 ..
-rw-r--r-- 1 aydin 197609 323544381 Jan  3 23:41 genome-scores.csv
-rw-r--r-- 1 aydin 197609     18103 Jan  3 23:41 genome-tags.csv
-rw-r--r-- 1 aydin 197609    570090 Jan  3 23:41 links.csv
-rw-r--r-- 1 aydin 197609   1397542 Jan  3 23:41 movies.csv
-rw-r--r-- 1 aydin 197609 533444411 Jan  3 23:41 ratings.csv
-rw-r--r-- 1 aydin 197609     10261 Jan  3 23:41 README.txt
-rw-r--r-- 1 aydin 197609  16603996 Jan  3 23:41 tags.csv


# Use Pandas to read data

In this notebook, we will be using three CSV files:

* ratings.csv: userId, movieId, rating, timestamp
* tags.csv: userId, movieId, tag, timestamp
* movies.csv: movieId, title, genres

In [2]:
import pandas as pd

In [3]:
movie_data = pd.read_csv('./movielens/movies.csv', sep=',')

In [4]:
movie_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
#Timestamps represent seconds since midnight Coordinated Universal Time (UTC)
tags = pd.read_csv('./movielens/tags.csv', sep=",")
tags.head()

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,1240597180
1,65,208,dark hero,1368150078
2,65,353,dark hero,1368150079
3,65,521,noir thriller,1368149983
4,65,592,dark hero,1368150078


In [6]:
ratings = pd.read_csv('./movielens/ratings.csv', sep=",")
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


### We do not need timestamp column right now but we will get back to it later on.

In [7]:
del ratings['timestamp']
del tags['timestamp']

# Descriptive Statistics

In [8]:
ratings['rating'].describe()

count    2.000026e+07
mean     3.525529e+00
std      1.051989e+00
min      5.000000e-01
25%      3.000000e+00
50%      3.500000e+00
75%      4.000000e+00
max      5.000000e+00
Name: rating, dtype: float64

In [9]:
ratings['rating'].mean()

3.5255285642993797

In [10]:
ratings['rating'].min(), ratings['rating'].max(), ratings['rating'].std()

(0.5, 5.0, 1.0519889192942424)

In [11]:
ratings['rating'].mode()

0    4.0
dtype: float64

## Quick sanity check

Here we are verifying adequacy of ratings by checking if there are any ratings greater than 5 or less than 0. It is unneccessary since we know min and max ratings values but this is an alternative way.

In [12]:
filter_1 = ratings['rating'] > 5

In [13]:
filter_1.any()

False

In [14]:
filter_2 = ratings['rating'] > 0

In [15]:
filter_2.all()

True

# Data Cleaning: Handling missing data

In [16]:
movie_data.shape

(27278, 3)

In [17]:
movie_data.isnull().any()

movieId    False
title      False
genres     False
dtype: bool

In [18]:
ratings.shape

(20000263, 3)

In [19]:
ratings.isnull().any()

userId     False
movieId    False
rating     False
dtype: bool

In [20]:
tags.shape

(465564, 3)

In [21]:
tags.isnull().any()

userId     False
movieId    False
tag         True
dtype: bool

In [22]:
null_ = tags['tag'].isnull()
null_

0         False
1         False
2         False
3         False
4         False
          ...  
465559    False
465560    False
465561    False
465562    False
465563    False
Name: tag, Length: 465564, dtype: bool

In [23]:
null_tags = tags[null_]
null_tags

,userId,movieId,tag
373276,116460,123,NaN
373277,116460,346,NaN
373281,116460,1184,NaN
373288,116460,1785,NaN
373289,116460,2194,NaN
373291,116460,2691,NaN
373299,116460,4103,NaN
373301,116460,4473,NaN
373303,116460,4616,NaN
373319,116460,7624,NaN


In [24]:
null_tag_movies = null_tags['movieId']
pd.DataFrame(movie_data, index=null_tag_movies)

,movieId,title,genres
movieId,,,
123,125.0,Flirting With Disaster (1996),Comedy
346,350.0,"Client, The (1994)",Drama|Mystery|Thriller
1184,1210.0,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Sci-Fi
1785,1869.0,Black Dog (1998),Action|Thriller
2194,2279.0,Urban Legend (1998),Horror|Thriller
2691,2777.0,Cobra (1925),Drama
4103,4197.0,Real Life (1979),Comedy
4473,4568.0,Best of the Best (1989),Action
4616,4711.0,Theremin: An Electronic Odyssey (1993),Documentary


### Since we have null values for tags, let's drop them to have a cleaner dataset

In [25]:
tags = tags.dropna()

In [26]:
tags.isnull().any()

userId     False
movieId    False
tag        False
dtype: bool

In [27]:
tags.shape

(465548, 3)

### Extract year from title e.g (1995)

In [28]:
movie_data['year'] = movie_data['title'].str.extract(".*\((.*)\).*", expand=True)

In [29]:
movie_data.tail()

,movieId,title,genres,year
27273,131254,Kein Bund für's Leben (2007),Comedy,2007
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,2002
27275,131258,The Pirates (2014),Adventure,2014
27276,131260,Rentun Ruusu (2001),(no genres listed),2001
27277,131262,Innocence (2014),Adventure|Fantasy|Horror,2014


# Merge Data - average movie ratings over time


In [30]:
average_rating = ratings[['movieId', 'rating']].groupby('movieId', as_index=False).mean()
average_rating.tail()

,movieId,rating
26739,131254,4.0
26740,131256,4.0
26741,131258,2.5
26742,131260,3.0
26743,131262,4.0


In [31]:
joined = movie_data.merge(average_rating, on='movieId', how='inner')
joined.head()

,movieId,title,genres,year,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,3.921240
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,3.211977
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,3.151040
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,2.861393
4,5,Father of the Bride Part II (1995),Comedy,1995,3.064592


In [32]:
joined.head()

,movieId,title,genres,year,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,3.921240
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,3.211977
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,3.151040
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,2.861393
4,5,Father of the Bride Part II (1995),Comedy,1995,3.064592


## Specify Data to be used in Twitter API request

In [33]:
rating_mask = joined.rating  > 4
year_mask = joined.year > '1990'
genre_mask = joined.genres.str.contains("Fantasy") | joined.genres.str.contains("Animation")

In [34]:
final_data = joined[rating_mask & year_mask & genre_mask]
final_data = final_data.reset_index().drop(columns='index')

In [35]:
import numpy as np
titles = np.array(final_data.title)
for index, title in np.ndenumerate(titles):
    final_data.loc[index, 'title'] = title.split("(")[0]
final_data


,movieId,title,genres,year,rating
0,720,Wallace & Gromit: The Best of Aardman Animation,Adventure|Animation|Comedy,1996,4.109473
1,745,Wallace & Gromit: A Close Shave,Animation|Children|Comedy,1995,4.167315
2,1148,Wallace & Gromit: The Wrong Trousers,Animation|Children|Comedy|Crime,1993,4.181068
3,3000,Princess Mononoke,Action|Adventure|Animation|Drama|Fantasy,1997,4.096299
4,4993,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy,2001,4.137925
5,5618,Spirited Away,Adventure|Animation|Fantasy,2001,4.203810
6,5952,"Lord of the Rings: The Two Towers, The",Adventure|Fantasy,2002,4.107521
7,7153,"Lord of the Rings: The Return of the King, The",Action|Adventure|Drama|Fantasy,2003,4.142382
8,26718,Life On A String,Adventure|Drama|Fantasy|Musical,1991,5.000000
9,27328,Monday,Action|Comedy|Crime|Fantasy|Thriller,2000,4.500000


In [36]:
import numpy as np
final_data = final_data.drop(final_data[final_data.title.str.contains(", The")].index, inplace = False)
final_data = final_data.reset_index().drop(columns='index')
titles = np.array(final_data.title)
for index, title in np.ndenumerate(titles):
    final_data.loc[index, 'hashtags'] = "#" +  title.replace(" ", "")

In [37]:
movies_DF = final_data.copy()
final_data

,movieId,title,genres,year,rating,hashtags
0,720,Wallace & Gromit: The Best of Aardman Animation,Adventure|Animation|Comedy,1996,4.109473,#Wallace&Gromit:TheBestofAardmanAnimation
1,745,Wallace & Gromit: A Close Shave,Animation|Children|Comedy,1995,4.167315,#Wallace&Gromit:ACloseShave
2,1148,Wallace & Gromit: The Wrong Trousers,Animation|Children|Comedy|Crime,1993,4.181068,#Wallace&Gromit:TheWrongTrousers
3,3000,Princess Mononoke,Action|Adventure|Animation|Drama|Fantasy,1997,4.096299,#PrincessMononoke
4,5618,Spirited Away,Adventure|Animation|Fantasy,2001,4.203810,#SpiritedAway
5,26718,Life On A String,Adventure|Drama|Fantasy|Musical,1991,5.000000,#LifeOnAString
6,27328,Monday,Action|Comedy|Crime|Fantasy|Thriller,2000,4.500000,#Monday
7,31658,Howl's Moving Castle,Adventure|Animation|Fantasy|Romance,2004,4.066078,#Howl'sMovingCastle
8,48394,Pan's Labyrinth,Drama|Fantasy|Thriller,2006,4.037898,#Pan'sLabyrinth
9,60069,WALL·E,Adventure|Animation|Children|Romance|Sci-Fi,2008,4.038929,#WALL·E


# Twitter API access

In [38]:
# Dependencies

import os
import tweepy as tw
import json
from pprint import pprint
import config
#put api key in config as consumer_key, api secret key as consumer_secret, 
#access token as access_token, access token secret as access_token_secret
#to be able to run access this api on your end

In [39]:
auth = tw.OAuthHandler(config.consumer_key, config.consumer_secret)
auth.set_access_token(config.access_token, config.access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [40]:
def twitter_pull(hash):
    users_and_text = {}
    tweet_list = ''
    tweets = tw.Cursor(api.search,
        q=hash,
            lang="en").items(1)
    for tweet in tweets:
        users_and_text["hashtag"] = hash
        users_and_text["user_name"] = tweet.user.screen_name
        users_and_text["tweet"] = tweet.text
        users_and_text["location"] = tweet.user.location
#     users_and_text = [[tweet.user.screen_name, tweet.text, tweet.user.location] for tweet in tweets]
    return users_and_text
    

In [41]:
twits = []
for hash in final_data.hashtags:
    twits.append(twitter_pull(hash))

In [43]:
for tweet in twits:
    print(tweet)

{}
{}
{}
{'hashtag': '#PrincessMononoke', 'user_name': 'kamidoll_', 'tweet': 'this was supposed to be for womens day but Her....\n#StudioGhibli #PrincessMononoke https://t.co/I2xszLWhgb', 'location': '🇵🇭 '}
{'hashtag': '#SpiritedAway', 'user_name': 'fairychiminie', 'tweet': 'RT @ArisaChibara: Motel key tags preorders dropping soon~\n#cardcaptorsakura #spiritedaway #sailormoon https://t.co/VyeAM0IYGA', 'location': '24 | virgo | isfj'}
{}
{'hashtag': '#Monday', 'user_name': 'MBJ8388', 'tweet': "RT @USArmy: It's still #Monday, so here's some #MondayMotivation for you. \n\nSoldiers with @25thinfantrydivision conduct a buddy-team #livef…", 'location': 'THE NO DATE ZONE‼️ Keep moving'}
{}
{}
{'hashtag': '#WALL·E', 'user_name': 'TheGodOfPegana', 'tweet': 'From my "Roger in Love" video. Roger is a robot from the comic book "Roger et ses humains" by @MonsieurDream and… https://t.co/KoOdEqIlRJ', 'location': ''}
{'hashtag': '#LettheRightOneIn', 'user_name': 'roll_weird', 'tweet': 'Runners up:\n\n

#### As can be seen some of tweet requests return empty dictionary. The reason for this can be different factors. Not optimized hashtag or the original name of the movie is different from the name in IMDB which is actually a translation, etc.

## We will add all dictionaries to dataframe and assign null value for those with no data

In [48]:
#Create the twitter dataframe
hashtagsList = movies_DF["hashtags"].to_list()
twitter_DF = pd.DataFrame(hashtagsList)

In [49]:
for index, _dict in enumerate(twits):
    if bool(_dict):
        twitter_DF.loc[index, 'user_name'] = _dict['user_name']
        twitter_DF.loc[index, 'tweet'] = _dict['tweet']
        twitter_DF.loc[index, 'location'] = _dict['location']
    else:
        twitter_DF.loc[index, 'user_name'] = None
        twitter_DF.loc[index, 'tweet'] = None
        twitter_DF.loc[index, 'location'] = None

## Clean dataframe from those rows with None values

In [50]:
twitter_DF = twitter_DF.dropna()
twitter_DF = twitter_DF.rename(columns={0: "hashtags"})

# PostgreSQL Database Ingestion

In [51]:
from sqlalchemy import create_engine

In [52]:
engine = create_engine(f'postgresql://{config.postgres}/Movies_DB') #path to the database on your local system
#put username, password, and localhost address in your config file as the postgres variable

In [53]:
#check that the tables exist in our SQL database
engine.table_names()

['movies', 'twitter']

In [54]:
#Put the dataframes into the SQL tables
engine.execute("DELETE from movies")
movies_DF = movies_DF.rename(columns={"movieId": "movieid"})
movies_DF.to_sql(name='movies', con=engine, if_exists='append', index=False)

In [55]:
engine.execute("DELETE from twitter")
twitter_DF.to_sql(name='twitter', con=engine, if_exists='append', index=False)

In [56]:
pd.read_sql_query('select movies.title, movies.rating, twitter.hashtags, twitter.user_name, twitter.tweet, twitter.location from movies join twitter on movies.hashtags = twitter.hashtags', con=engine)

,title,rating,hashtags,user_name,tweet,location
0,Princess Mononoke,4.096299,#PrincessMononoke,StephenCordeir7,Just picked this up today 😊 #studioghibli #Pr...,
1,Spirited Away,4.203810,#SpiritedAway,HolyGohan,RT @itsfilmthusiast: Did you know that #Spirit...,"Houston, TX - USA"
2,Monday,4.500000,#Monday,dalewfinkjr8,"RT @BethFratesMD: ""It is what a man thinks of ...","Houston, TX"
3,WALL·E,4.038929,#WALL·E,TheGodOfPegana,"From my ""Roger in Love"" video. Roger is a robo...",
4,Let the Right One In,4.010526,#LettheRightOneIn,roll_weird,Runners up:\n\n#Supernatural #LetTheRightOneIn...,"Kepler, WV"
5,FLCL,4.006627,#FLCL,DentrezIngerso1,@KariWahlgren Do you miss playing these strong...,
6,Up,4.038266,#Up,jason_shawky,In the end we all become strangers and memorie...,"Alexandria, Egypt"
7,How to Train Your Dragon,4.000420,#HowtoTrainYourDragon,YTZOMBIE_AU,RT @Dreamworks: #HowToTrainYourDragon: The Hid...,"Sydney, New South Wales"
8,Toy Story 3,4.012974,#ToyStory3,GyoBy_brand,Mini BMX Bicycle Toy Excellent Finger Mountain...,
9,Alcina,4.250000,#Alcina,Brassylassy,RT @maireflavin: Just an average Monday then.....,


## The final table does not have all fields filled for location but this is beacuse that information is not included on twitter account of those users